In [35]:
import torch
import pickle
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
from copy import deepcopy
from tqdm import tqdm


from model import LR
from data import FairnessDataset, SyntheticDataset, GermanDataset, IncomeDataset
from ei_effort import Optimal_Effort, PGD_Effort
from ei_utils import *
from ei_model_test import EIModel, fair_batch_proxy, covariance_proxy

# warnings.filterwarnings('ignore')

In [36]:
def get_wandb(model):
    for module in model.layers:
        if hasattr(module, 'weight'):
            weights = module.weight.data[0]
        if hasattr(module, 'bias'):
            bias = module.bias.data
            
    theta = torch.cat((weights, bias), 0)
    return theta.numpy().round(2)

In [37]:
# dataset = SyntheticDataset(seed=0)
dataset = GermanDataset(seed=0)
# dataset = IncomeDataset(seed=1)

In [38]:
def append_res(d, lamb, train_alpha, alpha, acc, fair_loss, ei, ei_model):
    d['lambda'].append(lamb)
    d['train_alpha'].append(train_alpha)
    d['alpha'].append(alpha)
    d['accuracy'].append(acc)
    d['loss'].append(1-acc)
    d['fair_loss'].append(fair_loss)
    d['ei_disparity'].append(ei)
    d['ei_model'].append(ei_model)

In [39]:
def model_runner(dataset: FairnessDataset, hp: dict, results: dict):
    tau = 0.5
    
    if hp['optimal_effort']:
        effort = Optimal_Effort(hp['delta'])
    else:
        effort = PGD_Effort(hp['delta'])
    
    if hp['robust_training']:
        train_alpha = hp['alpha']
    else:
        train_alpha = 0.
    
    train_tensors, val_tensors, test_tensors = dataset.tensor(z_blind=hp['z_blind'])
    train_dataset = FairnessDataset(*train_tensors, dataset.imp_feats)
    val_dataset = FairnessDataset(*val_tensors, dataset.imp_feats)
    test_dataset = FairnessDataset(*test_tensors, dataset.imp_feats)
    
    model = LR(num_features=train_dataset.X.shape[1])
    ei_model = EIModel(
        model = model,
        proxy = hp['proxy'],
        effort = effort,
        tau = tau,
        warm_start = True)
    
    ei_model.train(
        train_dataset,
        lamb=hp['lambda'],
        alpha=train_alpha,
        lr=hp['learning_rate'],
        n_epochs=hp['n_epochs'],
        batch_size=1024,
        abstol=hp['pga_abstol']
        )
    
    Y_hat, Y_hat_max, fair_loss = ei_model.predict(test_dataset, alpha=hp['alpha'], abstol=hp['pga_abstol'])
    accuracy, ei_disparity = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), Y_hat, Y_hat_max, tau)
    append_res(results, hp['lambda'], train_alpha, hp['alpha'], accuracy, fair_loss.item(), ei_disparity, ei_model)

In [40]:
def run_tradeoff(dataset, hyper_params):
    hp = hyper_params.copy()
    results = pd.DataFrame()
    
    results = {'lambda': [], 'train_alpha': [], 'alpha': [], 'accuracy': [], 'loss': [], 'fair_loss': [], 'ei_disparity': [], 'ei_model': []}
    for robust_training in hyper_params['robust_training']:
        for delta in hyper_params['delta']:
            for lamb in hyper_params['lambda']:
                for alpha in hyper_params['alpha']:
                    hp['robust_training'] = robust_training
                    hp['delta'] = delta
                    hp['lambda'] = lamb
                    hp['alpha'] = alpha
                
                    model_runner(dataset, hp, results)
    
    return results

In [41]:
torch.manual_seed(0)
# Hyperparameters
hyper_params = {}
hyper_params['lambda'] = [1.]
hyper_params['delta'] = [1.] # synthetic is 0.5
hyper_params['alpha'] = [0.5]
hyper_params['learning_rate'] = 0.01
hyper_params['n_epochs'] = 200
hyper_params['proxy'] = covariance_proxy
hyper_params['pga_abstol'] = 1e-7
hyper_params['z_blind'] = False
hyper_params['optimal_effort'] = True
hyper_params['robust_training'] = [False, True]

# Run tradeoffs
results = run_tradeoff(dataset, hyper_params)

Training [alpha=0.00; lambda=1.00; delta=1.00]:  41%|████      | 82/200 [00:05<00:07, 16.27epochs/s]


KeyboardInterrupt: 

In [15]:
df = pd.DataFrame(results).dropna()
# df[['model', 'model_adv']] = df[['model', 'model_adv']].map(get_wandb)
df.sort_values(['lambda', 'alpha']).reset_index(drop=True)

,lambda,train_alpha,alpha,accuracy,loss,fair_loss,ei_disparity,ei_model
0,0.8,0.0,0.0,0.730,0.270,0.000095,0.041667,<ei_model_test.EIModel object at 0x287e1c510>
1,0.8,0.0,0.0,0.725,0.275,0.000019,0.046757,<ei_model_test.EIModel object at 0x28f627c50>
2,0.8,0.0,0.1,0.735,0.265,0.001279,0.285908,<ei_model_test.EIModel object at 0x28f624910>
3,0.8,0.1,0.1,0.715,0.285,0.001298,0.202634,<ei_model_test.EIModel object at 0x287e48850>
4,0.8,0.0,0.5,0.755,0.245,0.009398,0.307588,<ei_model_test.EIModel object at 0x28f610d90>
5,0.8,0.5,0.5,0.715,0.285,0.008445,0.333333,<ei_model_test.EIModel object at 0x17e4ed6d0>
6,0.8,0.0,1.5,0.730,0.270,0.024988,0.456522,<ei_model_test.EIModel object at 0x287e4add0>
7,0.8,1.5,1.5,0.725,0.275,0.028035,0.460638,<ei_model_test.EIModel object at 0x28f6138d0>
8,0.8,0.0,5.0,0.715,0.285,0.039882,0.487805,<ei_model_test.EIModel object at 0x17e4f5550>
9,0.8,5.0,5.0,0.735,0.265,0.041934,0.449921,<ei_model_test.EIModel object at 0x1758f4550>


In [34]:
df = pd.DataFrame(results)
# df[['model', 'model_adv']] = df[['model', 'model_adv']].map(get_wandb)
df.sort_values(['lambda', 'alpha']).reset_index(drop=True)

,lambda,train_alpha,alpha,accuracy,loss,fair_loss,ei_disparity,ei_model
0,1.0,0.0,0.0,0.39275,0.60725,NaN,0.00000,<ei_model_test.EIModel object at 0x2990ce250>
1,1.0,0.0,0.0,0.39275,0.60725,NaN,0.00000,<ei_model_test.EIModel object at 0x298e4fb10>
2,1.0,0.0,0.5,0.66100,0.33900,8.386072e-16,0.43335,<ei_model_test.EIModel object at 0x28f627090>
3,1.0,0.5,0.5,0.39275,0.60725,NaN,0.00000,<ei_model_test.EIModel object at 0x299065d10>
4,1.0,0.0,5.0,0.39275,0.60725,NaN,0.00000,<ei_model_test.EIModel object at 0x28f651750>
5,1.0,5.0,5.0,0.39275,0.60725,NaN,0.00000,<ei_model_test.EIModel object at 0x287c763d0>


In [27]:
for ei_model in results['ei_model']:
    px.scatter(vars(ei_model.train_history), y='f_loss').show()

In [24]:
for ei_model in results['ei_model']:
    px.scatter(vars(ei_model.train_history), y='p_loss').show()